# Modificación dinamica de un objeto

In [1]:
class A:
    def __init__(self,n):
        self.x = n
    
a = A(2)
a.__setattr__('y',50)
a.__dict__['z']='lol'

print(f"{a.x=} {a.y=} {a.z=}")
print(f"las variables de a: {vars(a)}")

a.x=2 a.y=50 a.z='lol'
las variables de a: {'x': 2, 'y': 50, 'z': 'lol'}


In [2]:
class Persona:
    def __init__(self, nombre):
        self.nombre = nombre


luisito = Persona("Luis")

# oh no! me olvide de que pueda saludar

def saludar(self):
    return f"Hola a todos! soy {self.nombre}"

Persona.saludar = saludar

print(luisito.saludar())
print(f"las variables de luisito: {vars(luisito)}")
print(f"luisito tiene el metodo saludar? {'si' if hasattr(luisito,'saludar') else 'no'}")



Hola a todos! soy Luis
las variables de luisito: {'nombre': 'Luis'}
luisito tiene el metodo saludar? si


# Agregar comportamiendo a tipos built-in

In [3]:
class Entero(int):
    def __new__(cls, valor, *args, **kwargs):
        instancia = super().__new__(cls, valor)
        instancia.mensaje = f"Hola a todos! soy el entero {valor}"
        return instancia

num = Entero(5)
print(num)
print(num.mensaje)


5
Hola a todos! soy el entero 5


# ¡Cuidado con mutables como default argument!
### Hoy todos los IDEs decente alertan sobre esto y no es una sorpresa para los programadores de python pero si lo es para los nuevos en el lenguaje.

In [4]:
def f(a, L=[]):
    i = 0
    while i <= a:
        L.append(i**2)
        i += 1
    return L

uno = f(1)
print(uno)
dos = f(2)
tres = f(3)
print(uno); print(dos); print(tres) #si, python soporta ; para separar sentencias. Mala practica


[0, 1]
[0, 1, 0, 1, 4, 0, 1, 4, 9]
[0, 1, 0, 1, 4, 0, 1, 4, 9]
[0, 1, 0, 1, 4, 0, 1, 4, 9]


# Redondeo
Python usa para redondear el "metodo del banquero" o "redondeo a par". Es distinto a muchos lenguajes como C (aunque similar a Pascal, Lisp)

In [5]:
numeros = [-2.5, -1.5, -.5, .5, 1.5, 2.5, 3.5, 4.5, 5.5]
for numero in numeros:
    print(f"{numero} redondeado es: {round(numero)}")


-2.5 redondeado es: -2
-1.5 redondeado es: -2
-0.5 redondeado es: 0
0.5 redondeado es: 0
1.5 redondeado es: 2
2.5 redondeado es: 2
3.5 redondeado es: 4
4.5 redondeado es: 4
5.5 redondeado es: 6


Podemos analizar como es en C usando libm disponible en linux para evitar compilar un ejemplo

In [6]:
import ctypes
from ctypes.util import find_library

libm = ctypes.CDLL(find_library('m'))
libm.round.argtypes = [ctypes.c_double]
libm.round.restype = ctypes.c_double

for numero in numeros:
    print(f"{numero} redondeado es: {libm.round(float(numero))}")


-2.5 redondeado es: -3.0
-1.5 redondeado es: -2.0
-0.5 redondeado es: -1.0
0.5 redondeado es: 1.0
1.5 redondeado es: 2.0
2.5 redondeado es: 3.0
3.5 redondeado es: 4.0
4.5 redondeado es: 5.0
5.5 redondeado es: 6.0


# Operador %
El operador de módulo (%) en C y Python difiere en su manejo de números negativos: en C, el residuo hereda el signo del dividendo (basado en truncado hacia cero), lo que puede dar resultados negativos, mientras que en Python sigue el signo del divisor (con división floor), asegurando un residuo no negativo cuando el divisor es positivo. Esta sutil discrepancia es crucial al portar código o simular comportamientos entre lenguajes


In [7]:
# en python tendriamos este comportamiento
print([x % 10 for x in range(-10,11)])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0]


In [8]:
# en C tenemos este comportamiento
# nota: este ejemplo de archivo temporales para crear un archivo con el snipet de C y compilarlo me lo sugirió Grok.
# me parecio ingenioso aunque sea sobre ingenieria. 
# requiere gcc 


import subprocess
import tempfile
import os

# se escapa '\'
c_code = '''
#include <stdio.h>

int main(void) {
  int x;

  printf("[");

  for (x = -10; x < 11; x++) {
    printf("%d", (x % 10));
    x != 10 ? printf(" ,") : printf("]\ \n");
  };

  return 0;
};
'''

with tempfile.NamedTemporaryFile(mode='w', suffix='.c', delete=False) as f:
    f.write(c_code)
    c_filename = f.name

try:
    compile_result = subprocess.run(['gcc', c_filename, '-o', 'modulo'], 
                                    capture_output=True, text=True)
    if compile_result.returncode != 0:
        print("Error de compilación:", compile_result.stderr)
    else:
        run_result = subprocess.run(['./modulo'], capture_output=True, text=True)
        if run_result.returncode == 0:
            print(str(run_result.stdout))
        else:
            print("Error de ejecución:", run_result.stderr)
finally:
    os.unlink(c_filename)
    if os.path.exists('modulo'):
        os.unlink('modulo')


[0 ,-9 ,-8 ,-7 ,-6 ,-5 ,-4 ,-3 ,-2 ,-1 ,0 ,1 ,2 ,3 ,4 ,5 ,6 ,7 ,8 ,9 ,0]


Se puede emular el comportamiento de **%** en C con este codigo

In [9]:
from math import trunc

def c_rem(dividend: int | float, divisor: int) -> int | float:
    """Return remainder like C: r = a - n* trunc(a/n)"""
    if divisor == 0:
        raise ValueError("math domain error")
    return dividend - divisor * trunc(dividend / divisor)

print([c_rem(x, 10) for x in range(-10,11)])

[0, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0]


# XOR para buscar un item faltante
*Se te da una array A de n - 1 números enteros que están en el rango entre 1 y n. Todos los números aparecen exactamente una vez, excepto uno, que falta. Encuentra este número que falta.*

In [10]:
from random import randint
from functools import reduce
from operator import xor

# Tenemos una lista grande de numeros consecutivos
lista_grande = list(range(1,1500))

# Tenemos una lista A a la que le falta un elemento

A = lista_grande.copy()
A.remove(randint(1, 1499))

print(f"el elemento de que falta es: {reduce(xor,lista_grande) ^ reduce(xor, A)}")

# resultado que podemos confirmar haciendo diferencia de conjuntos
print(f"el elemento de que falta usando conjuntos es: {set(lista_grande) - set(A)}")


el elemento de que falta es: 1434
el elemento de que falta usando conjuntos es: {1434}


# ¡Cuidado con la notacion de tuplas implicita!

In [11]:
resultado = 4 * 0,5
print(resultado)

(0, 5)
